# Pycaret 및 rulefit 설치
설치후 런타임 - 세션 재시작 하고 한글폰트 설치 코드부터 시행

In [ ]:
!pip install -U pip setuptools wheel
!pip install -U scikit-learn imbalanced-learn
!pip install -U "git+https://github.com/pycaret/pycaret.git@master"
!pip install -U jinja2
!pip install rulefit
!pip install catboost
!pip install scikit_posthocs

# 한글 폰트 설치(상관 히트맵 용)

In [ ]:
import sys

# Google Colab 환경에서 실행 중인지 확인
if 'google.colab' in sys.modules:
    # debconf를 Noninteractive 모드로 설정
    !echo 'debconf debconf/frontend select Noninteractive' | \
    debconf-set-selections

    # fonts-nanum 패키지를 설치
    !sudo apt-get -qq -y install fonts-nanum

    # Matplotlib의 폰트 매니저 가져오기
    import matplotlib.font_manager as fm

    # 나눔 폰트의 시스템 경로 찾기
    font_files = fm.findSystemFonts(fontpaths=['/usr/share/fonts/truetype/nanum'])

    # 찾은 각 나눔 폰트를 Matplotlib 폰트 매니저에 추가
    for fpath in font_files:
        fm.fontManager.addfont(fpath)

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import pearsonr
from scipy.stats import skew, kurtosis
#from pycaret.classification import *
#from pycaret.classification import get_config
#from rulefit import RuleFit
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold



plt.rcParams['font.family'] = 'NanumGothic' # 한글 불러오기
plt.rcParams['axes.unicode_minus'] = False

# 데이터 불러오기

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/novicedata/Paper/refs/heads/main/MZ_turnover/data/gong2024.csv') # 데이터 불러오기
df

## 역문항 및 차원 축소

In [ ]:
def preprocessing_binary(df):
    print(f'raw data: {df.shape} \n')

    print(f'MZ data 확인 \n')
    df = df[df['dq2'] >= 1981].copy() # MZ세대 데이터만 추출

    print(f'MZ data: {df.shape} \n')

    print(f'불필요 데이터 삭제 \n')
    remove_col = ['ID', 'q1_1_m1', 'q1_1_m2', 'q5_1', 'q5_2', 'q5_3', 'q5_1_1', 'q5_2_1', 'q5_3_1',
                 'q6_1', 'q6_2', 'q6_3', 'q6_4', 'q6_1_1', 'q6_2_1', 'q6_3_1', 'q6_4_1', 'a',
                 'q9_m1', 'q9_m2', 'q9_m3',
                 'q17_m1', 'q17_m2', 'q18_m1', 'q18_m2', 'q28_m1', 'q28_m2', 'q28_m3', 'b',
                 'q31_C1', 'q31_C2', 'q38_1', 'q38_2_m1', 'q38_2_m2',
                 'dq1', 'DM3', 'DM4', 'DM5', 'DM6', 'DM7', 'WT', 'TWT']

    df = df.drop(remove_col, axis=1)
    print(f'삭제 후 shape: {df.shape} \n')

    # 역문항 수동 처리
    print(f'역문항 처리 \n')
    columns_to_reverse = ['q8_5', 'q14_4', 'q19_5', 'q33_5', 'q33_6', 'q33_7', 'q33_8']

    def reverse(x):
        if x == 5:
            return 1
        elif x == 4:
            return 2
        elif x == 3:
            return 3
        elif x == 2:
            return 4
        else:
            return 5

    for column in columns_to_reverse:
        df[column] = df[column].apply(reverse)

    new_data = df.copy()

    # 문항 요인화(평균으로)
    print(f'유사 문항 평균화 \n')
    new_data['업무량'] = new_data[['q1']]
    new_data['근무환경'] = new_data[['q2']]
    new_data['자원제공'] = new_data[['q3_1', 'q3_2', 'q3_3']].mean(axis=1)
    new_data['업무자율성'] = new_data[['q4_1', 'q4_2', 'q4_3']].mean(axis=1)
    new_data['채용'] = new_data[['q7_1', 'q7_2', 'q7_3', 'q7_4']].mean(axis=1)
    new_data['승진'] = new_data[['q8_1', 'q8_2', 'q8_3', 'q8_4', 'q8_5']].mean(axis=1)
    new_data['배치전환'] = new_data[['q10_1', 'q10_2', 'q10_3']].mean(axis=1)
    new_data['보수 및 보상'] = new_data[['q11_1', 'q11_2', 'q11_3', 'q11_4']].mean(axis=1)
    new_data['호봉제'] = new_data[['q12']]
    new_data['후생복지'] = new_data[['q13_1', 'q13_2']].mean(axis=1)
    new_data['교육훈련/능력발전'] = new_data[['q14_1', 'q14_2', 'q14_3', 'q14_4']].mean(axis=1)
    new_data['업무 수행 역량'] = new_data[['q15_1', 'q15_2', 'q15_3']].mean(axis=1)
    new_data['전문성 수준'] = new_data[['q16_1', 'q16_2']].mean(axis=1)
    new_data['조직목표'] = new_data[['q19_1', 'q19_2', 'q19_3', 'q19_4', 'q19_5']].mean(axis=1)
    new_data['리더십'] = new_data[['q20_1', 'q20_2', 'q20_3', 'q20_4', 'q20_5', 'q20_6', 'q20_7']].mean(axis=1)
    new_data['과업지향 조직문화'] = new_data[['q21_1', 'q21_2']].mean(axis=1)
    new_data['혁신지향 조직문화'] = new_data[['q21_3', 'q21_7']].mean(axis=1)
    new_data['관계지향 조직문화'] = new_data[['q21_4', 'q21_8']].mean(axis=1)
    new_data['위계질서 조직문화'] = new_data[['q21_5', 'q21_6']].mean(axis=1)
    new_data['조직 냉소주의'] = new_data[['q21_9', 'q21_10']].mean(axis=1)
    new_data['조직 내 정치'] = new_data[['q21_11', 'q21_12']].mean(axis=1)
    new_data['의사결정'] = new_data[['q22_1', 'q22_2', 'q22_3', 'q22_4']].mean(axis=1)
    new_data['변화관리'] = new_data[['q23_1', 'q23_2', 'q23_3']].mean(axis=1)
    new_data['협업/의사소통'] = new_data[['q24_1', 'q24_2', 'q24_3']].mean(axis=1)
    new_data['조직성과'] = new_data[['q25_1', 'q25_2']].mean(axis=1)
    new_data['성과관리'] = new_data[['q26_1', 'q26_2', 'q26_3', 'q26_4', 'q26_5']].mean(axis=1)
    new_data['업무성과'] = new_data[['q27_1', 'q27_2', 'q27_3']].mean(axis=1)
    new_data['부정부패'] = new_data[['q29']]
    new_data['공공봉사동기'] = new_data[['q30_1', 'q30_2', 'q30_3', 'q30_4', 'q30_5', 'q30_6']].mean(axis=1)
    new_data['역할갈등'] = new_data[['q31_1', 'q31_2', 'q31_3']].mean(axis=1)
    new_data['역할모호성'] = new_data[['q31_4', 'q31_5', 'q31_6']].mean(axis=1)
    new_data['업무 과중'] = new_data[['q31_7']]
    new_data['조직시민행동'] = new_data[['q32_1', 'q32_2', 'q32_3', 'q32_4', 'q32_5']].mean(axis=1)
    new_data['혁신행동'] = new_data[['q33_1', 'q33_2', 'q33_3', 'q33_4', 'q33_5', 'q33_6', 'q33_7', 'q33_8']].mean(axis=1)
    new_data['공직가치'] = new_data[['q34_1', 'q34_2', 'q34_3', 'q34_4', 'q34_5', 'q34_6', 'q34_7', 'q34_8']].mean(axis=1)
    new_data['조직몰입'] = new_data[['q35_1', 'q35_2', 'q35_3', 'q35_4']].mean(axis=1)
    new_data['직무만족'] = new_data[['q36_1', 'q36_2', 'q36_3']].mean(axis=1)
    new_data['공직만족도'] = new_data[['q37_1', 'q37_2', 'q37_3', 'q37_4']].mean(axis=1)
    new_data['삶의 질'] = new_data[['q39_1', 'q39_2']].mean(axis=1)
    new_data['성별'] = new_data[['DM2']]
    new_data['출생 연도'] = new_data[['dq2']]
    new_data['혼인상태'] = new_data[['dq3']]
    new_data['전체 자녀의 수'] = new_data[['dq4_1']]
    new_data['만 20세 미만 자녀의 수'] = new_data[['dq4_2']]
    new_data['입직당시 학력'] = new_data[['dq5_1']]
    new_data['현재 학력'] = new_data[['dq5_2']]
    new_data['입직 준비기간'] = new_data[['dq6']]
    new_data['재직기간'] = new_data[['dq7']]
    new_data['입직 시 채용 방식'] = new_data[['dq8']]
    new_data['입직당시 직급'] = new_data[['dq9_1']]
    new_data['현재 직급'] = new_data[['dq9_2']]
    new_data['1개월 평균 급여수준'] = new_data[['dq10']]
    new_data['1주일 평균 근무시간'] = new_data[['dq11']]
    new_data['기관유형'] = new_data[['orgtype']]
    new_data['기관구분'] = new_data[['DM1']]


    # 평균화 후, 요인만 남기고 문항 제거
    print(f'불필요 데이터 삭제 2 \n')

    remove_col2 = [
        'DM2', 'dq2', 'dq3', 'dq4_1', 'dq4_2', 'dq5_1', 'dq5_2',
        'dq6', 'dq7', 'dq8', 'dq9_1', 'dq9_2', 'dq10', 'dq11', 'orgtype', 'DM1',
        'q1', 'q2',
                   "q3_1", "q3_2", "q3_3",
      "q4_1", "q4_2", "q4_3",
      "q7_1", "q7_2", "q7_3", "q7_4",
      "q8_1", "q8_2", "q8_3", "q8_4", "q8_5",
      "q10_1", "q10_2", "q10_3",
      "q11_1", "q11_2", "q11_3", "q11_4", 'q12',
      "q13_1", "q13_2",
      "q14_1", "q14_2", "q14_3", "q14_4",
      "q15_1", "q15_2", "q15_3", 'q16_1', 'q16_2',
      "q19_1", "q19_2", "q19_3", "q19_4", "q19_5",
      "q20_1", "q20_2", "q20_3", "q20_4", "q20_5", "q20_6", "q20_7",
      'q21_1', 'q21_2', 'q21_3', 'q21_5', 'q21_6', 'q21_7', 'q21_4', 'q21_8', 'q21_9', 'q21_10', 'q21_11', 'q21_12',
      "q22_1", "q22_2", "q22_3", "q22_4",
      "q23_1", "q23_2", "q23_3",
      "q24_1", "q24_2", "q24_3",
      "q25_1", "q25_2",
      "q26_1", "q26_2", "q26_3", "q26_4", "q26_5",
      "q27_1", "q27_2", "q27_3", 'q29',
      "q30_1", "q30_2", "q30_3", "q30_4", "q30_5", "q30_6",
      "q31_1", "q31_2", "q31_3", "q31_4", "q31_5", "q31_6", "q31_7",
      "q32_1", "q32_2", "q32_3", "q32_4", "q32_5",
      "q33_1", "q33_2", "q33_3", "q33_4", "q33_5", "q33_6", "q33_7", "q33_8",
      "q34_1", "q34_2", "q34_3", "q34_4", "q34_5", "q34_6", "q34_7", "q34_8",
      "q35_1", "q35_2", "q35_3", "q35_4",
      "q36_1", "q36_2", "q36_3",
      "q37_1", "q37_2", "q37_3", "q37_4",
      "q39_1", "q39_2"]

    new_data = new_data.drop(remove_col2, axis=1)
    print(f'불필요 칼럼 처리2 : {new_data.shape}', '\n')

    # 종속변수인 이직의도 이진화
    print(f'이직의도 이진화 \n')
    condition1 = [
        (df['q38'] <=3),
        (df['q38'] > 3)
    ]
    choice1 = [0, 1]
    new_data.loc[:, '이직의도'] = np.select(condition1, choice1, default = np.nan)
    new_data = new_data.drop(['q38'], axis=1)

    return new_data

In [ ]:
data = preprocessing_binary(df) # 원데이터에서 전처리

In [ ]:
# 재직기간별 이직의도 비율 계산
turnover_by_tenure = (
    data.groupby('재직기간')['이직의도']
    .mean()  # 1의 비율 = 이직의도 비율
    .reset_index()
    .rename(columns={'이직의도': '이직의도 비율'})
)

# 보기 좋게 정렬
turnover_by_tenure = turnover_by_tenure.sort_values('재직기간')
print(turnover_by_tenure)

# 1 (5년 이하), 2 (6~10년), 3 (11~15년),
# 4 (16~20년), 5 (21~25년), 6 (26년 이상)

# 기술 통계

## 크론바흐 알파

In [ ]:
# 크론바흐 알파
def cronbach_alpha(df):
    df = df.dropna()
    item_vars = df.var(axis=0, ddof=1)
    total_var = df.sum(axis=1).var(ddof=1)
    n_items = df.shape[1]
    return round(n_items / (n_items - 1) * (1 - item_vars.sum() / total_var), 3)

# 역문항 목록
columns_to_reverse = ['q8_5', 'q14_4', 'q19_5', 'q33_5', 'q33_6', 'q33_7', 'q33_8']

# 역문항 처리 (1~5 리커트)
df_reversed = df.copy()
for col in columns_to_reverse:
    if col in df_reversed.columns:
        df_reversed[col] = df_reversed[col].apply(lambda x: 6 - x if pd.notnull(x) else np.nan)

# 측정하고 싶은 변수 집합 (원문항 기준)
scales = {
    '자원제공': ['q3_1', 'q3_2', 'q3_3'],
    '업무자율성': ['q4_1', 'q4_2', 'q4_3'],
    '채용': ['q7_1', 'q7_2', 'q7_3', 'q7_4'],
    '승진': ['q8_1', 'q8_2', 'q8_3', 'q8_4', 'q8_5'],
    '배치전환': ['q10_1', 'q10_2', 'q10_3'],
    '보수 및 보상': ['q11_1', 'q11_2', 'q11_3', 'q11_4'],
    '후생복지': ['q13_1', 'q13_2'],
    '교육훈련/능력발전': ['q14_1', 'q14_2', 'q14_3', 'q14_4'],
    '업무 수행 역량': ['q15_1', 'q15_2', 'q15_3'],
    '전문성 수준': ['q16_1', 'q16_2'],
    '조직목표': ['q19_1', 'q19_2', 'q19_3', 'q19_4', 'q19_5'],
    '리더십': ['q20_1', 'q20_2', 'q20_3', 'q20_4', 'q20_5', 'q20_6', 'q20_7'],
    '과업지향 조직문화': ['q21_1', 'q21_2'],
    '혁신지향 조직문화': ['q21_3', 'q21_7'],
    '관계지향 조직문화': ['q21_4', 'q21_8'],
    '위계질서 조직문화': ['q21_5', 'q21_6'],
    '조직 냉소주의': ['q21_9', 'q21_10'],
    '조직 내 정치': ['q21_11', 'q21_12'],
    '의사결정': ['q22_1', 'q22_2', 'q22_3', 'q22_4'],
    '변화관리': ['q23_1', 'q23_2', 'q23_3'],
    '협업/의사소통': ['q24_1', 'q24_2', 'q24_3'],
    '조직성과': ['q25_1', 'q25_2'],
    '성과관리': ['q26_1', 'q26_2', 'q26_3', 'q26_4', 'q26_5'],
    '업무성과': ['q27_1', 'q27_2', 'q27_3'],
    '공공봉사동기': ['q30_1', 'q30_2', 'q30_3', 'q30_4', 'q30_5', 'q30_6'],
    '역할갈등': ['q31_1', 'q31_2', 'q31_3'],
    '역할모호성': ['q31_4', 'q31_5', 'q31_6'],
    '조직시민행동': ['q32_1', 'q32_2', 'q32_3', 'q32_4', 'q32_5'],
    '혁신행동': ['q33_1', 'q33_2', 'q33_3', 'q33_4', 'q33_5', 'q33_6', 'q33_7', 'q33_8'],
    '공직가치': ['q34_1', 'q34_2', 'q34_3', 'q34_4', 'q34_5', 'q34_6', 'q34_7', 'q34_8'],
    '조직몰입': ['q35_1', 'q35_2', 'q35_3', 'q35_4'],
    '직무만족': ['q36_1', 'q36_2', 'q36_3'],
    '공직만족도': ['q37_1', 'q37_2', 'q37_3', 'q37_4'],
    '삶의 질': ['q39_1', 'q39_2']
}


# 결과 저장
results = []

for name, items in scales.items():
    valid_items = [col for col in items if col in df_reversed.columns]
    if len(valid_items) < 2:
        results.append({'변수': name, '문항 수': len(valid_items), 'Alpha': '문항 부족'})
    else:
        alpha_val = cronbach_alpha(df_reversed[valid_items])
        results.append({
            '변수': name,
            '문항 수': len(valid_items),
            'Alpha': round(alpha_val, 3) if not np.isnan(alpha_val) else '계산 불가'
        })

# 결과 출력
alpha_df = pd.DataFrame(results)
alpha_df = alpha_df.sort_values(by='변수').reset_index(drop=True)
print(alpha_df)


## 최대, 최소, 중앙, 평균, 표준편차, 왜도, 첨도

In [ ]:
# 범주형 변수 리스트
categorical_vars = [
    '성별', '혼인상태',
    '입직당시 학력', '현재 학력',
    '입직 준비기간', '재직기간', '입직 시 채용 방식',
    '입직당시 직급', '현재 직급',
    '1개월 평균 급여수준', '1주일 평균 근무시간',
    '기관유형', '기관구분', '이직의도'
]

# 수치형 변수 추출
numeric_data = data.drop(columns=categorical_vars)

# 최빈값 계산 함수
def get_mode(series):
    return series.mode().iloc[0] if not series.mode().empty else None

# 1. 수치형 변수 기술 통계 요약 (왜도, 첨도 포함)
summary = pd.DataFrame({
    'min': numeric_data.min(),
    'max': numeric_data.max(),
    'mode': numeric_data.apply(get_mode),
    'mean': numeric_data.mean(),
    'std': numeric_data.std(),
    'skewness': numeric_data.apply(skew),
    'kurtosis': numeric_data.apply(lambda x: kurtosis(x, fisher=True) + 3)
}).round(3)

sorted_columns = sorted(summary.index, key=lambda x: x)
summary = summary.reindex(sorted_columns).round(3)

print("수치형 변수 기술 통계")
display(summary)


# 2. 범주형 변수 빈도표 출력
for col in categorical_vars:
    print(f"\n▶ 변수: {col}")
    freq = data[col].value_counts(dropna=False)
    percent = data[col].value_counts(normalize=True, dropna=False) * 100

    freq_table = pd.DataFrame({
        '빈도': freq,
        '비율(%)': percent.round(2)
    })

    print(freq_table)

summary.to_csv('summary_stats.csv', encoding='utf-8-sig')

## 상관

In [ ]:
# 데이터 불러오기(수치형)
df_numeric = data.select_dtypes(include='number')
cols = df_numeric.columns
corr = df_numeric.corr().round(3)

# 상관계수 및 p-value 저장
pval_matrix = pd.DataFrame(index=cols, columns=cols, dtype=float)
stars_matrix = pd.DataFrame(index=cols, columns=cols, dtype=str)

# p-value 계산 및 별 표기
for col1 in cols:
    for col2 in cols:
        _, pval = pearsonr(df_numeric[col1], df_numeric[col2])
        pval_matrix.loc[col1, col2] = pval
        if pval <= 0.001:
            star = '***'
        elif pval <= 0.01:
            star = '**'
        elif pval <= 0.05:
            star = '*'
        else:
            star = ''
        stars_matrix.loc[col1, col2] = star

mask = np.triu(np.ones_like(corr, dtype=bool))  # 상삼각 마스킹

# 히트맵 시각화 (색상은 상관계수로, 글씨는 별만 표시)
corr = df_numeric.corr()
plt.figure(figsize=(20, 18))
sns.heatmap(corr, annot=stars_matrix, mask=mask, fmt='', cmap='coolwarm', center=0, linewidths=0.5,
           annot_kws = {'fontweight' : "bold"})
plt.xticks(fontsize=10, fontweight='bold')
plt.yticks(fontsize=10, fontweight='bold')
plt.tight_layout()
plt.savefig("correlation_heatmap.png", dpi=1200)
plt.show()


# 모델 적합(pycaret)

In [ ]:
# 10-fold × 5 repeats (총 25 분할)
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)

# 동일 전처리/시드
clf = setup(
    data=data,                    # 전처리된 데이터프레임
    target='이직의도',           # 종속변수 (0/1)
    session_id=42,              # 재현성
    fold_strategy=cv,           # 동일 분할기 공유
    fold=10,                     # 내부 표기(무시되지 않도록 유지)
    verbose=False
)

candidates = [
'lr', 'nb', 'knn', 'catboost',
    'dt', 'rf', 'gbc', 'xgboost',
    'lightgbm'
]

# 각 모델 동일조건 튜닝(AUC 기준)
tuned_models = []
for code in candidates:
    base = create_model(code, cross_validation=True)  # 1차 성능 확인
    tuned = tune_model(
        base, optimize='AUC',
        n_iter=10,
        choose_better=True,     # 개선 시 교체
        fold=cv
    )
    tuned_models.append(tuned)

# 재평가(per-fold 원점수 추출)
all_results = []
for m in tuned_models:
    _ = create_model(m, cross_validation=True, fold=cv)
    res = pull().copy()                 # 분할별 메트릭 테이블
    res['model'] = m.__class__.__name__
    all_results.append(res)

res_df = pd.concat(all_results, ignore_index=True)

# 모델별 평균/SD/SE 계산 (SE = SD/sqrt(50))
metrics = ['Accuracy','AUC','Recall','Prec.','F1']
summary = (res_df
           .groupby('model')[metrics]
           .agg(['mean','std'])
          )
for met in metrics:
    summary[(met, 'SE')] = summary[(met, 'std')] / np.sqrt(50)

# 보기 좋게 열 정리
summary = summary.swaplevel(0,1,axis=1).sort_index(axis=1)


In [ ]:
summary['mean']

In [ ]:
summary['std']

In [ ]:
summary['SE']

In [ ]:
# AUC 기준 최고 성능 보인 random forest 모델 튜닝 (10 iter)
from sklearn.model_selection import RepeatedStratifiedKFold
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=5, random_state=42)
clf = setup(
    data=data,
    target='이직의도',
    session_id=42,
    fold_strategy=cv,
    fold=10,
    verbose=False
)
rf_model = create_model('rf')
tuned_rf = tune_model(
    rf_model,
    optimize = 'AUC',
    choose_better = True,
    n_iter = 10
)

# RuleFit 규칙 추출

In [ ]:
tuned_rf

In [ ]:
# 앞서 random forest 적합할 때 사용한 train, test set 추출
## random forest 적합 시 사용한 데이터만을 rulefit으로 규칙을 추출해야 하기 때문.

X_train = get_config('X_train')
y_train = get_config('y_train')
X_test = get_config('X_test')
y_test = get_config('y_test')

In [ ]:
rf = RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight={},
                       criterion='entropy', max_depth=9, max_features='sqrt',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0, min_samples_leaf=6,
                       min_samples_split=10, min_weight_fraction_leaf=0.0,
                       monotonic_cst=None, n_estimators=230, n_jobs=-1,
                       oob_score=False, random_state=42, verbose=0,
                       warm_start=False) # 앞서 적합한 random forest의 파라미터

rulefit = RuleFit(tree_generator = rf, rfmode='classification', random_state = 19990423) # rulefit setup

In [ ]:
# rulefit 알고리즘에 따라 rule 추출 및 규칙 성능 지표 계산
rulefit.fit(X_train.values, y_train.values, feature_names=X_train.columns)

In [ ]:
rules = rulefit.get_rules()
rules

### importance

절댓값 계수와 support의 변동성을 곱한 값

규칙이 널리 적용되면서도 충분한 분류력을 갖고 있을 때 중요도가 높아짐.

coef만 크거나 support만 크거나 해서 생기는 편향 방지 지표

coef와 support를 모두 사용하는 지표로 균형잡힌 중요도 지표

In [ ]:
# 규칙(rule)만 필터링하여 중요도 기준 정렬
rules_rules = rules[rules['type'] == 'rule'].copy()

# 중요도 기준 내림차순 정렬 및 importance 0 제거
rules_importance = (
    rules_rules
    .sort_values('importance', ascending=False)
    .query('importance != 0')
    .round(3)
    .reset_index(drop=True)
)

# CSV 저장
rules_importance.to_csv('rules_rules_only.csv', encoding='utf-8-sig', index=False)

# 결과 확인
rules_importance


In [ ]:
rules_imp = rules[(rules['type'] == 'rule') & (rules['importance'] > 0)].sort_values('importance', ascending=False).reset_index(drop=True)
imp_values = rules_imp['importance'].values

# --- elbow index 계산 ---
elbow_idx = 4
elbow_value = imp_values[elbow_idx]

# --- 그래프 시각화 ---
plt.figure(figsize=(8,5))
plt.plot(range(1, len(imp_values)+1), imp_values, marker='o', markersize=4, label='Importance')
plt.axvline(elbow_idx+1, color='red', linestyle='--', label=f'Elbow point (Rule {elbow_idx+1})')
plt.scatter(elbow_idx+1, elbow_value, color='red', s=60, zorder=5)
plt.title('규칙 중요도 (내림차순)')
plt.xlabel('규칙 순위')
plt.ylabel('중요도')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"꺾임점 위치: {elbow_idx+1}번째 규칙, 중요도 값 ≈ {elbow_value:.3f}")


In [ ]:
num = 0

# importance 기준 top 5의 규칙, 중요도, 회귀계수, support 추출
for i, j in rules_importance.iterrows():
    if num == 5:
        continue
    else:
        print(f"rule: {j['rule']}")
        print(f"importance: {j['importance']}")
        print(f"coef: {j['coef']}")
        print(f"support: {j['support']}", '\n')

        num += 1